In [ ]:
from __future__ import print_function
import os
from os import listdir
from os.path import join, isfile, isdir
from shutil import copyfile, rmtree
from PIL import Image

RAW_PATH = './Face'
DATA_PATH = './data'
TRAIN = 'Train'
TEST = 'Test'
IMAGE_SIZE = 32, 32

In [ ]:
from sklearn.svm import SVC
import numpy as np
from imageio import imread

In [ ]:
data_path = "./data/"
path_type = "Train"
images_path = join(data_path, path_type + "_resized")
images_path

In [ ]:
train_images_path = [join(images_path, f) for f in listdir(images_path) if isfile(join(images_path, f)) and 'jpg' in f]
len(train_images_path)

In [ ]:
train_images, train_labels, test_images, test_labels = [], [], [], []

In [ ]:
for train_image_path in train_images_path:
    image = imread(train_image_path)
    image = image.reshape(32*32)
    image_label = train_image_path.split('/')[-1].split('_')[0]
    train_images.append(image)
    train_labels.append(image_label)


In [ ]:
print(list(set(train_labels)))

In [ ]:
img = imread('test.jpg')
img = img.reshape(186*141*3)
img = [i/255. for i in img]
img.shape

In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "cnn"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

In [3]:
import tensorflow as tf

height = 28
width = 28
channels = 1
n_inputs = height * width

conv1_fmaps = 32
conv1_ksize = 3
conv1_stride = 1
conv1_pad = "SAME"

conv2_fmaps = 64
conv2_ksize = 3
conv2_stride = 2
conv2_pad = "SAME"

pool3_fmaps = conv2_fmaps

n_fc1 = 64
n_outputs = 27

reset_graph()

with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X")
    X_reshaped = tf.reshape(X, shape=[-1, height, width, channels])
    y = tf.placeholder(tf.int32, shape=[None], name="y")

conv1 = tf.layers.conv2d(X_reshaped, filters=conv1_fmaps, kernel_size=conv1_ksize,
                         strides=conv1_stride, padding=conv1_pad,
                         activation=tf.nn.relu, name="conv1")
conv2 = tf.layers.conv2d(conv1, filters=conv2_fmaps, kernel_size=conv2_ksize,
                         strides=conv2_stride, padding=conv2_pad,
                         activation=tf.nn.relu, name="conv2")

with tf.name_scope("pool3"):
    pool3 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")
    pool3_flat = tf.reshape(pool3, shape=[-1, pool3_fmaps * 7 * 7])

with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(pool3_flat, n_fc1, activation=tf.nn.relu, name="fc1")

with tf.name_scope("output"):
    logits = tf.layers.dense(fc1, n_outputs, name="output")
    Y_proba = tf.nn.softmax(logits, name="Y_proba")

with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

In [9]:
import os
from os import listdir
from os.path import isfile, join
import numpy as np
from imageio import imread

DATA_PATH = './data'
LABELS = [
    'nguyennt3', 'hieubq', 'trangdnt', 'diepdt2', 'tiennd', 'phuongnt3',
    'nhanlt3', 'dathv', 'thanhnt', 'tuannm4', 'anhpt4', 'ChiHoaLaoCong',
    'anhnp2', 'anhnh2', 'thanhpv2', 'truongtd4', 'huyencm', 'thanhlc',
    'hadn', 'hieuvm', 'thaontp2', 'thaonp2', 'chaudtb', 'thuybt', 'nhungvth',
    'vinhnd2', 'anhpq']

def read_data(path_type):
    if not path_type:
        return
    data_path = join(DATA_PATH, path_type + "_resized")
    images_path = [
        join(data_path, f) for f in listdir(data_path)
        if f.endswith('.jpg')
    ]
    images, labels = [], []
    for image_path in images_path:
        image = imread(image_path)
        image = image.reshape(28*28)
        image = [i/255. for i in image]
        image_label = LABELS.index(image_path.split('/')[-1].split('_')[0])
        images.append(image)
        labels.append(image_label)
    return np.array(images), np.array(labels)

def shuffle_data(images, labels):
    l = images.shape[0]
    idx = np.arange(l)
    np.random.shuffle(idx)
    return images[idx, ], labels[idx, ]

In [5]:
train_images, train_labels = read_data('Train')
test_images, test_labels = read_data('Test')
train_images.shape, train_labels.shape

((17102, 784), (17102,))

In [13]:
n_epochs = 50
batch_size = 250
n_iterations = train_images.shape[0] // batch_size

In [ ]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        shuffle_data(train_images, train_labels)
        train_imgs = train_images[:n_iterations*batch_size, ]
        train_lbs = train_labels[:n_iterations*batch_size, ]
        X_train = np.split(train_imgs, n_iterations)
        y_train = np.split(train_lbs, n_iterations)
        for iteration in range(n_iterations):
            X_batch, y_batch = X_train[iteration], y_train[iteration]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: test_images, y: test_labels})
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)

        save_path = saver.save(sess, "./my_mnist_model")

0 Train accuracy: 0.432 Test accuracy: 0.348873
1 Train accuracy: 0.676 Test accuracy: 0.522234
2 Train accuracy: 0.788 Test accuracy: 0.575102
3 Train accuracy: 0.82 Test accuracy: 0.607172
4 Train accuracy: 0.872 Test accuracy: 0.633299
5 Train accuracy: 0.904 Test accuracy: 0.651025
6 Train accuracy: 0.912 Test accuracy: 0.657787
7 Train accuracy: 0.928 Test accuracy: 0.664447
8 Train accuracy: 0.944 Test accuracy: 0.668443
9 Train accuracy: 0.944 Test accuracy: 0.675205
10 Train accuracy: 0.944 Test accuracy: 0.682787
11 Train accuracy: 0.952 Test accuracy: 0.686475
12 Train accuracy: 0.948 Test accuracy: 0.686475
13 Train accuracy: 0.952 Test accuracy: 0.687193
14 Train accuracy: 0.952 Test accuracy: 0.686168
15 Train accuracy: 0.96 Test accuracy: 0.684016
16 Train accuracy: 0.972 Test accuracy: 0.682787
17 Train accuracy: 0.964 Test accuracy: 0.683402
18 Train accuracy: 0.968 Test accuracy: 0.682275
19 Train accuracy: 0.96 Test accuracy: 0.686066
20 Train accuracy: 0.968 Test acc